<a href="https://colab.research.google.com/github/DavideEva/Moviefy/blob/main/Moviefy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem analysis

# Download datasets

In [ ]:
from glob import glob

In [ ]:
films_data = {
    "Ghibli": "1RR18MAxLoZQWsxrmfb1hYif2MhOcsgO2"
}
keys = list(films_data.keys())
ghibli_index = 0
films_data[keys[ghibli_index]]

In [ ]:
def load_data(name, id_txt):

  file_name = f'list-{name}.txt'

  ! gdown --id "$id_txt" -O "$file_name"

  lines = []
  with open(file_name, "r") as f:
    lines = f.readlines()
  
  ! mkdir "$name"

  for line in lines:
    id = line.strip()
    ! cd "$name" && gdown --id "$id"

  zip_files = glob(f'{name}/*.zip')
  for zip_file in zip_files:
    ! unzip -qo "$zip_file" -d "$name"
    ! rm "$zip_file"
  
  return name

In [ ]:
folders = [load_data(studio_name, id_list_id) for studio_name, id_list_id in films_data.items()]

# Import

In [ ]:
import numpy as np
import math
import random
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Layer, InputSpec, LeakyReLU, Input, Conv2D, Activation, Concatenate, Conv2DTranspose, BatchNormalization, AveragePooling2D, Add
from tensorflow import pad
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Plot functions

In [ ]:
def plot_grid(images, columns, show_axis=False, labels=None):
  if len(images) == 0 or columns <= 0:
    return
  height = 1 + math.ceil(len(images) / columns) * 2
  width = columns * 4
  dpi = max(images[0].shape[0], images[0].shape[1]) // 2
  fig = plt.figure(figsize=(width, height), dpi=dpi)
  fig.subplots_adjust(hspace=0.4)
  for index, img in enumerate(images, start=1):
    if 'float' in img.dtype.str:
      img = (img * 255).astype('uint8')
    sp = fig.add_subplot(math.ceil(len(images) / columns), columns, index)
    if not show_axis:
      plt.axis('off')
    plt.imshow(img)
    if labels is not None:
      l = len(labels)
      sp.set_title(labels[(index-1) % l], fontsize=10)
    else:
      sp.set_title(index, fontsize=10)

# Global parameters

In [ ]:
raw_size = (1080, 1920, 3)
input_shape = (224, 224, 3)
batch_size = 32
validation_split = 0.2

# Dataset loading and preprocessing

In [ ]:
def smooth_edges(img):
  return img

In [ ]:
data_generator_settings = {
    'data_format' : 'channels_last',
    'validation_split' : validation_split,
    'rescale' : 1.0 / 255
}

data_flow_settings = {
    'target_size' : target_size,
    'color_mode' : 'rgb',
    'class_mode' : None,
    'batch_size' : batch_size,
    'shuffle' : True,
    'interpolation' : 'bilinear'
}

cartoon_real_generator = ImageDataGenerator(
    **data_generator_settings
)

cartoon_edge_fake_generator = ImageDataGenerator(
    **data_generator_settings,
    preprocessing_function = smooth_edges
)

# Same proportions as raw, same height as desired input
target_size = (input_shape[0], raw_size[1] * input_shape[0] // raw_size[0])

In [ ]:
def random_cropper_generator(batches, size):
  def random_crop(img):
    left = bool(random.getrandbits(1))
    if left:
      return img[:, 0:size[1], :]
    return img[:, -size[1]:, :]
  for batch in batches:
    yield [random_crop(i) for i in batch]

In [ ]:
test_cartoon_real_flow = random_cropper_generator(
  cartoon_real_generator.flow_from_directory(
    **data_flow_settings,
    directory = folders[ghibli_index],
    subset = 'training',
  ),
  input_shape
)
plot_grid(next(test_cartoon_real_flow), 4)

In [ ]:
test_cartoon_edge_fake_flow = random_cropper_generator(
  cartoon_edge_fake_generator.flow_from_directory(
    **data_flow_settings,
    directory = folders[ghibli_index],
    subset = 'training',
  ),
  input_shape
)
plot_grid(next(test_cartoon_edge_fake_flow), 4)
del test_cartoon_edge_fake_flow

# Cartoon-GAN

## Utility Layers

In [ ]:
class ReflectionPadding2D(Layer):
  def __init__(self, padding=(1, 1), **kwargs):
    self.padding = tuple(padding)
    # self.input_spec = [InputSpec(ndim=4)]
    super(ReflectionPadding2D, self).__init__(**kwargs)

  def compute_output_shape(self, s):
    if s[1] == None:
      return (None, None, None, s[3])
    return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

  def call(self, x, mask=None):
    w_pad, h_pad = self.padding
    return tf.pad(x, [[0, 0], [h_pad, h_pad], [w_pad, w_pad], [0, 0]], 'REFLECT')

  def get_config(self):
    config = super(ReflectionPadding2D, self).get_config()
    return config

In [ ]:
class Conv2DReflection3x3(Layer):
  def __init__(self, features, stride=1):
    super().__init__()
    self.reflectionPadding2D = ReflectionPadding2D()
    self.conv2d = Conv2D(features, (3,3), strides=(stride, stride), padding='valid', use_bias=False)

  def call(self, inputs, training=False):
    x = self.reflectionPadding2D(inputs, training=training)
    return self.conv2d(x, training=training)

## Discriminator

In [ ]:
# define the discriminator model
def define_discriminator(image_shape):
  alpha = 0.2
  epsilon = 1e-5
  momentum = 0.1

  # source image input
  in_image = Input(shape=image_shape)

  # k3n32s1
  d = Conv2DReflection3x3(32, stride=1)(in_image)
  d = LeakyReLU(alpha=alpha)(d)

  # k3n64s2
  d = Conv2DReflection3x3(64, stride=2)(d)
  d = LeakyReLU(alpha=alpha)(d)
  # k3n128s1
  d = Conv2DReflection3x3(128, stride=1)(d)
  d = BatchNormalization(epsilon=epsilon, momentum=momentum)(d)
  d = LeakyReLU(alpha=alpha)(d)

  # k3n128s2
  d = Conv2DReflection3x3(128, stride=2)(d)
  d = LeakyReLU(alpha=alpha)(d)
  # k3n256s1
  d = Conv2DReflection3x3(256, stride=1)(d)
  d = BatchNormalization(epsilon=epsilon, momentum=momentum)(d)
  d = LeakyReLU(alpha=alpha)(d)

  # feature construction block
  # k3n256s1
  d = Conv2DReflection3x3(256, stride=1)(d)
  d = BatchNormalization(epsilon=epsilon, momentum=momentum)(d)
  d = LeakyReLU(alpha=alpha)(d)

  # patch output
  patch_out = Conv2DReflection3x3(1, stride=1)(d)

  # define model
  model = Model(in_image, patch_out)
  return model

In [ ]:
D = define_discriminator(input_shape)

In [ ]:
plot_model(D, show_shapes=True, expand_nested=True)

## Generator

In [ ]:
# define the generator model
def define_generator(image_shape):
  alpha = 0.2
  epsilon = 1e-5
  momentum = 0.1

  # source image input
  in_image = Input(shape=image_shape)

  # flat block
  # k7n64s1
  g = Conv2D(64, (7,7), strides=1, padding='same', use_bias=False)(in_image)
  g = BatchNormalization(epsilon=epsilon, momentum=momentum)(g)
  g = LeakyReLU(alpha=alpha)(g)

  def down_block(x, n_features):
    # k3n?s2
    x = Conv2DReflection3x3(n_features, stride=2)(x)
    # k3n?s1
    x = Conv2DReflection3x3(n_features, stride=1)(x)
    x = BatchNormalization(epsilon=epsilon, momentum=momentum)(x)
    x = LeakyReLU(alpha=alpha)(x)
    return x

  # 1st down block
  g = down_block(g, 128)

  # 2nd down block
  g = down_block(g, 256)

  def resiual_block(x):
    skip = x
    # k3n256s1
    x = Conv2DReflection3x3(256, stride=1)(x)
    x = BatchNormalization(epsilon=epsilon, momentum=momentum)(x)
    x = LeakyReLU(alpha=alpha)(x)
    # k3n256s1
    x = Conv2DReflection3x3(256, stride=1)(x)
    x = BatchNormalization(epsilon=epsilon, momentum=momentum)(x)
    x = Add()([x, skip])
    x = LeakyReLU(alpha=alpha)(x)
    return x

  for _ in range(8):
    g = resiual_block(g)

  def up_block(x, n_features):
    # k3n?s1/2
    x = Conv2DTranspose(n_features, (3,3), strides=2)(x)
    x = AveragePooling2D(pool_size=(2,2), strides=1)(x)
    # k3n?s1
    x = Conv2DReflection3x3(n_features, stride=1)(x)
    x = BatchNormalization(epsilon=epsilon, momentum=momentum)(x)
    x = LeakyReLU(alpha=alpha)(x)
    return x

  # 1st up block
  g = up_block(g, 128)

  # 2nd up-block
  g = up_block(g, 64)

  # k7n3s1
  output = Conv2D(3, (7,7), strides=1, padding='same')(g)

  # define model
  model = Model(in_image, output)
  return model

In [ ]:
G = define_generator(input_shape)

In [ ]:
plot_model(G, show_shapes=True, expand_nested=True)

## Loss functions

In [ ]:
def BCEWithLogitsLoss():
  return tf.keras.losses.BinaryCrossEntropy(
    from_logits=True,
    reduction=tf.keras.losses.Reduction.NONE)

In [ ]:
class AdversarialLoss:
  def __init__(self, cartoon_labels, fake_cartoon_labels):
    self.base_loss = BCEWithLogitsLoss()
    self.cartoon_labels = cartoon_labels
    self.fake_cartoon_labels = fake_cartoon_labels

  def __call__(self, cartoon, generated_fake, cartoon_edge_fake):
    D_cartoon_loss = self.base_loss(cartoon, self.cartoon_labels)
    D_generated_fake_loss = self.base_loss(generated_fake, self.fake_cartoon_labels)
    D_edge_fake_loss = self.base_loss(cartoon_edge_fake, self.fake_cartoon_labels)

    return D_cartoon_loss + D_generated_fake_loss + D_edge_fake_loss

# alias for clarity
DiscriminatorLoss = AdversarialLoss

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)

class ContentLoss:
  def __init__(self):
    self.perception = vgg19.predict
  
  def __call__(self, outputs, inputs):
    diff = self.perception(outputs) - self.perception(inputs)
    k = tf.norm(diff, ord=1)
    return k

In [ ]:
class GeneratorLoss:
  def __init__(self, omega=10):
    self.omega = omega
    self.content_loss = ContentLoss()
    self.base_loss = BCEWithLogitsLoss()
  
  def __call__(self, outputs, inputs):
    return self.base_loss(outputs, inputs) + self.omega * self.content_loss(outputs, inputs)